In [1]:
import pandas as pd
import datetime
import warnings
import numpy as np
warnings.simplefilter("ignore")

In [2]:
cities_res = ['asheville', 'boston', 'chicago', 'dc', 'denver', 'montreal', 'nashville', 'ri', 'sf', 'tc']
cities_db = ['asheville', 'boston', 'chicago', 'dc', 'denver', 'montreal', 'nashville', 'quebec', 'sf', 'tc']
path_root = 'processed_data/'

In [16]:
def clean_price(s):
    if type(s) == str:
        return float(s.replace(",","").replace("$",""))
    elif type(s) == float:
        return s

def clean_data(df):
    df = df[df.minimum_nights_prev <= 28]
    df['price'] = df.price_prev.apply(clean_price)
    df['date'] = pd.to_datetime(df.date)
    df = df.sort_values("date", ascending=True)
    df = df[df.date <= datetime.datetime(2019,12,29)]
    df = df[df.date >= datetime.datetime(2019,2,18)] # First date for February
    return df

In [17]:
def process_month(city, end):
    df = pd.read_csv(path_root + city + end)
    df = clean_data(df)
    df['month'] = df.date.dt.month
    df = df.groupby('month').agg({"price": "mean", 'id': "count"})
    df['unit'] = city
    df = df.reset_index()
    if city == 'boston':
        df['trt'] = np.where(df.month < 7, 0, 1)
    else:
        df['trt'] = 0
    df = df.rename({'id': 'nights'}, axis = 1)
    
    return df

In [18]:
month_res_list = [process_month(city, '_reservation_date.csv') for city in cities_res]
by_month_res = pd.concat(month_res_list)
by_month_res.to_csv("long_reservation_date_by_month.csv", index = False)

In [19]:
def process_week(city, end):
    df = pd.read_csv(path_root + city + end)
    df = clean_data(df)
    df['month'] = df.date.dt.month
    df['week'] = df.date.dt.week
    df = df.groupby('week').agg({"price": "mean", 'id': "count", 'month':"first"})
    df['unit'] = city
    df = df.reset_index()
    if city == 'boston':
        df['trt'] = np.where(df.month < 7, 0, 1)
    else:
        df['trt'] = 0
    df = df.rename({'id': 'nights'}, axis = 1)
    df = df[df.week >= 8]
    return df

In [7]:
week_res_list = [process_week(city, '_reservation_date.csv') for city in cities_res]
by_week_res = pd.concat(week_res_list)
by_week_res.to_csv("long_reservation_date_by_week.csv", index = False)

In [156]:
date_list = pd.Series([datetime.datetime(2019,2,17) + datetime.timedelta(days=x) for x in range(0, 317, 14)])
date_list = pd.DataFrame({"date": date_list, "biweek_no": range(1, len(date_list)+1)})


def process_bimonth(city, end):
    df = pd.read_csv(path_root + city + end)
    df = clean_data(df)
    df['month'] = df.date.dt.month
    df['week'] = df.date.dt.week
    df['bimonth'] = pd.cut(df.date, date_list.date, labels=np.arange(1,23))
    df = df.groupby('bimonth').agg({"price": "mean", 'id': "count", 'month':"first", 'week': "first"})
    df['unit'] = city
    df = df.reset_index()
    if city == 'boston':
        df['trt'] = np.where(df.month < 7, 0, 1)
    else:
        df['trt'] = 0
    df = df.rename({'id': 'nights'}, axis = 1)
    df = df.dropna()
    df = df[df.bimonth != 10] # Remove the during period 
    return df

In [157]:
sm_res_list = [process_bimonth(city, '_reservation_date.csv') for city in cities_res]
by_sm_res = pd.concat(sm_res_list)

In [158]:
by_sm_res.to_csv("long_reservation_date_by_sm_final.csv", index = False)

In [151]:
norm = pd.read_csv("date_normalization.csv")
norm['dates'] = ['jul_2018_aug_2018', 'aug_2018_sep_2018', 'sep_2018_oct_2018', 'oct_2018_nov_2018', 'nov_2018_dec_2018', 
                 'dec_2018_jan_2019', 'jan_2019_feb_2019', 'feb_2019_mar_2019', 'mar_2019_apr_2019', 'apr_2019_may_2019',
                'may_2019_jun_2019']
norm['period_no'] = np.arange(1, norm.shape[0] + 1)
norm_mapper = {}
for city in cities_db:
    norm_mapper[city] = {}
    for date, days in zip(norm.dates, norm[city]):
        norm_mapper[city][date] = days
number_mapper = {}
for date, period in zip(norm.dates, norm.period_no):
    number_mapper[date] = period

In [152]:
def clean_data(df):
    # df = df[df.minimum_nights_prev <= 28]
    df['price'] = df.price_prev.apply(clean_price)
    df['date'] = pd.to_datetime(df.date)
    df = df.sort_values("date", ascending=True)
    return df

In [153]:
def process(city, end):
    df = pd.read_csv(path_root + city + end)
    df = clean_data(df)
    df = df.groupby('reservation_period').agg({"price": "mean", 'id': "count"})
    df['unit'] = city
    df = df.reset_index()
    df = df.merge(norm, left_on = 'reservation_period', right_on = 'dates', how = 'right')
    df = df.rename({'id': 'nights'}, axis = 1)
#     df['period_no'] = df.reservation_period.map(number_mapper)
    df['unit'] = df.unit.fillna(city)
    df['nights_diff'] = df.dates.map(norm_mapper[city])
    df['price'] = df.price.fillna(0)
    df['nights'] = df.nights.fillna(0)
    df = df.drop(["reservation_period"], axis = 1)
    if city == 'boston':
        df['trt'] = np.where(df.period_no < 6, 0, 1)
    else:
        df['trt'] = 0
    df = df[df.period_no != 6]
    
    df['nights_per_period'] = df.nights / df.nights_diff
    df = df.sort_values('period_no')
    return df

In [154]:
process('dc', '_date_booked.csv')

price  nights unit              dates  asheville  boston  chicago  \
5   157.670103     485   dc  jul_2018_aug_2018         25      30       30   
1   178.951754    1596   dc  aug_2018_sep_2018         25      28       28   
10  174.389175     388   dc  sep_2018_oct_2018         31      27       27   
9   194.567134     998   dc  oct_2018_nov_2018         34      37       35   
8   226.786382    1498   dc  nov_2018_dec_2018         26      26       28   
4   176.355641    6515   dc  jan_2019_feb_2019         21      23       23   
3   169.117558   13202   dc  feb_2019_mar_2019         36      31       31   
6   140.372130   21342   dc  mar_2019_apr_2019         29      34       34   
0   153.468179   26506   dc  apr_2019_may_2019         33      34       34   
7   157.189293   27740   dc  may_2019_jun_2019         31      26       26   

    dc  denver  montreal  nashville  quebec  sf  tc  period_no  nights_diff  \
5   31      27        35         34      35  32  32          1           31   
1   27      25        29         28      29  33  33          2           27   
10  28      27        28         28      28  25  25          3           28   
9   34      35        32         35      28  31  31          4           34   
8   28      29        30         27      32  33  33          5           28   
4   25      20        23         23      23  23  22          7           25   
3   29      38        31         30      29  33  33          8           29   
6   34      30        33         34      34  28  28          9           34   
0   35      32        32         31      33  30  32         10           35   
7   27      31        26         27      24  30  28         11           27   

    trt  nights_per_period  
5     0          15.645161  
1     0          59.111111  
10    0          13.857143  
9     0          29.352941  
8     0          53.500000  
4     0         260.600000  
3     0         455.241379  
6     0         627.705882  
0     0         757.314286  
7     0        1027.407407

In [155]:
db_list = [process(city, '_date_booked.csv') for city in cities_db]
db = pd.concat(db_list)
db.to_csv("long_date_booked2.csv", index = False)

In [119]:
db

reservation_period       price  nights       unit  period_no  num_nights  \
5   jul_2018_aug_2018  245.733871     248  asheville          1          25   
1   aug_2018_sep_2018  149.670270     185  asheville          2          25   
10  sep_2018_oct_2018  257.192171     281  asheville          3          31   
9   oct_2018_nov_2018  250.135417     384  asheville          4          34   
8   nov_2018_dec_2018  226.880769     520  asheville          5          26   
..                ...         ...     ...        ...        ...         ...   
4   jan_2019_feb_2019   86.333333      24         tc          7          22   
3   feb_2019_mar_2019  129.333333     165         tc          8          33   
6   mar_2019_apr_2019  234.226415     106         tc          9          28   
0   apr_2019_may_2019  177.876777     211         tc         10          32   
7   may_2019_jun_2019  142.129979     477         tc         11          28   

    trt  nights_per_period  
5     0           9.920000  
1     0           7.400000  
10    0           9.064516  
9     0          11.294118  
8     0          20.000000  
..  ...                ...  
4     0           1.090909  
3     0           5.000000  
6     0           3.785714  
0     0           6.593750  
7     0          17.035714  

[88 rows x 8 columns]

In [110]:
norm

dates  asheville  boston  chicago  dc  denver  montreal  \
0   jul_2018_aug_2018         25      30       30  31      27        35   
1   aug_2018_sep_2018         25      28       28  27      25        29   
2   sep_2018_oct_2018         31      27       27  28      27        28   
3   oct_2018_nov_2018         34      37       35  34      35        32   
4   nov_2018_dec_2018         26      26       28  28      29        30   
5   dec_2018_jan_2019         42      35       35  35      39        35   
6   jan_2019_feb_2019         21      23       23  25      20        23   
7   feb_2019_mar_2019         36      31       31  29      38        31   
8   mar_2019_apr_2019         29      34       34  34      30        33   
9   apr_2019_may_2019         33      34       34  35      32        32   
10  may_2019_jun_2019         31      26       26  27      31        26   

    nashville  quebec  sf  tc  period_no  
0          34      35  32  32          1  
1          28      29  33  33          2  
2          28      28  25  25          3  
3          35      28  31  31          4  
4          27      32  33  33          5  
5          36      37  34  35          6  
6          23      23  23  22          7  
7          30      29  33  33          8  
8          34      34  28  28          9  
9          31      33  30  32         10  
10         27      24  30  28         11